In [4]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV

In [5]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [6]:
os.getcwd()

'/opt/program'

In [7]:
# load model from 2020-06-10 notebook
bundle = joblib.load('/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib')

In [8]:
bundle

{'notebook': '2020-06-10-again',
 'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=3,
               min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, objective='multi:softprob', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1),
 'actuals': array([60, 60, 60, ...,  2,  2,  2]),
 'predictions': array([46, 46, 46, ...,  2,  2,  2]),
 'confusion_matrix': array([[ 193,   64,   14, ...,    0,    0,  150],
        [  27, 1167,   16, ...,    0,    0,   28],
        [   4,    6,  379, ...,    0,    0,  236],
        ...,
        [   0,    0,   17, ...,    0,    0,   46],
        [  67,  599,   24, ...,    0,    0,   78],
        [  19,    9,  196, ...,    0,    0, 2017]]),
 'walltime_train': '19min 45s',
 'pre

In [20]:
# quick manual accuracy computation
bundle['actuals'].shape, bundle['predictions'].shape
correct = len([i for i, _ in enumerate(bundle['actuals'])
              if bundle['actuals'][i] == bundle['predictions'][i]
              ])
print({'overall accuracy': correct/len(bundle['actuals']), 
       'num_correct': correct, 'total': len(bundle['actuals'])})
print(bundle['confusion_matrix'].shape)

confusion_diagonal = [bundle['confusion_matrix'][i][j]
                                  for i in range(54)
                                  for j in range(54)
                                  if i == j]
print(confusion_diagonal)
print('confusion, sum, diagonal', sum(confusion_diagonal))

{'overall accuracy': 0.1541161182619253, 'num_correct': 64992, 'total': 421708}
(54, 54)
[193, 1167, 379, 0, 0, 1406, 0, 1039, 30505, 0, 685, 557, 67, 62, 49, 0, 4534, 0, 2583, 109, 0, 0, 2766, 2003, 0, 0, 0, 236, 0, 2722, 0, 0, 612, 6064, 581, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 4632, 0, 0, 0, 0, 0, 0, 2017]
confusion, sum, diagonal 64992


In [25]:
model = bundle['model']
print(len(model.feature_importances_))
print(sorted(model.feature_importances_))

78
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00057195826, 0.00096085493, 0.0019978876, 0.0028608162, 0.003014325, 0.0031509278, 0.005542396, 0.0055443724, 0.0056226645, 0.0064963517, 0.0066661313, 0.007307959, 0.007578683, 0.0077737863, 0.008284873, 0.00866166, 0.009315344, 0.010423137, 0.010517005, 0.010585468, 0.010727769, 0.012014797, 0.012143584, 0.012632706, 0.01298739, 0.013684955, 0.014033192, 0.014308239, 0.014311185, 0.016608043, 0.017402876, 0.017734889, 0.017809916, 0.017948298, 0.018543119, 0.01898507, 0.019225849, 0.02069202, 0.020770036, 0.021067692, 0.02152036, 0.022673236, 0.024081945, 0.02418218, 0.02421692, 0.026871085, 0.027934289, 0.02844202, 0.030187974, 0.032351416, 0.033994444, 0.035514593, 0.03653184, 0.036576174, 0.044999287, 0.04772858, 0.05568539]


In [23]:
# 78 features
model.classes_

array([ 0,  1,  2,  4,  5,  7,  8, 11, 12, 13, 14, 15, 17, 18, 19, 21, 23,
       24, 25, 26, 27, 30, 32, 33, 34, 36, 37, 39, 40, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68,
       71, 72, 73])

In [29]:
import fresh.utils as fu
from importlib import reload
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [31]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
# = sorted(stationsdf.neighborhood.unique().tolist())



In [35]:
len(bundle['preproc']['one_hot_enc'].categories[0]), len(bundle['preproc']['one_hot_enc'].categories[1])

(75, 3)

In [36]:
features = ([f'feature_start_neighborhood={x}' for x in bundle['preproc']['one_hot_enc'].categories[0]]
           + [f'feature_gender={x}' for x in bundle['preproc']['one_hot_enc'].categories[1]])
print(features)

['feature_start_neighborhood=Alphabet City', 'feature_start_neighborhood=Battery Park City', 'feature_start_neighborhood=Bedford-Stuyvesant', 'feature_start_neighborhood=Bloomingdale', 'feature_start_neighborhood=Boerum Hill', 'feature_start_neighborhood=Bowery', 'feature_start_neighborhood=Broadway Triangle', 'feature_start_neighborhood=Brooklyn Heights', 'feature_start_neighborhood=Brooklyn Navy Yard', 'feature_start_neighborhood=Carnegie Hill', 'feature_start_neighborhood=Carroll Gardens', 'feature_start_neighborhood=Central Park', 'feature_start_neighborhood=Chelsea', 'feature_start_neighborhood=Chinatown', 'feature_start_neighborhood=Civic Center', 'feature_start_neighborhood=Clinton Hill', 'feature_start_neighborhood=Cobble Hill', 'feature_start_neighborhood=Columbia Street Waterfront District', 'feature_start_neighborhood=Downtown Brooklyn', 'feature_start_neighborhood=Dumbo', 'feature_start_neighborhood=East Harlem', 'feature_start_neighborhood=East Village', 'feature_start_nei

In [39]:
importances = model.feature_importances_
labeled_importances = [[features[i], importances[i]] for i in range(78)]
sorted(labeled_importances, key=lambda x:x[1])

[['feature_start_neighborhood=Bloomingdale', 0.0],
 ['feature_start_neighborhood=Broadway Triangle', 0.0],
 ['feature_start_neighborhood=Carnegie Hill', 0.0],
 ['feature_start_neighborhood=Carroll Gardens', 0.0],
 ['feature_start_neighborhood=Cobble Hill', 0.0],
 ['feature_start_neighborhood=East Harlem', 0.0],
 ['feature_start_neighborhood=East Williamsburg', 0.0],
 ['feature_start_neighborhood=Governors Island', 0.0],
 ['feature_start_neighborhood=Gowanus', 0.0],
 ['feature_start_neighborhood=Greenpoint', 0.0],
 ['feature_start_neighborhood=Hunters Point', 0.0],
 ['feature_start_neighborhood=Lenox Hill', 0.0],
 ['feature_start_neighborhood=Long Island City', 0.0],
 ['feature_start_neighborhood=Prospect Heights', 0.0],
 ['feature_start_neighborhood=Prospect Park', 0.0],
 ['feature_start_neighborhood=Red Hook', 0.0],
 ['feature_start_neighborhood=Stuyvesant Heights', 0.0],
 ['feature_start_neighborhood=Sunset Park', 0.0],
 ['feature_start_neighborhood=Upper East Side', 0.0],
 ['feature

In [ ]:
# Perhaps indeed the starting locations which are so un-important, basically have just way too many
# destinations, or maybe there are just not enough trips involving those stations.
# But anyway, I think ultimately the importances which are there 
#    across different cross validation folds are the ones to focus on

In [40]:
# retry that multi-logloss ...
rng = np.random.RandomState(31337)
indices1 = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    indices1.append([train_index, test_index])

indices2 = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    indices2.append([train_index, test_index])
    


In [44]:
# hmm doesnt look the same though, guess that random seed doesnt work deterministically?
indices1[0][:5] , indices2[0][:5]


([array([     0,      2,      3, ..., 843411, 843412, 843415]),
  array([     1,      7,      8, ..., 843410, 843413, 843414])],
 [array([     5,      7,      9, ..., 843412, 843413, 843414]),
  array([     0,      1,      2, ..., 843408, 843411, 843415])])

In [51]:
#log_loss(y_true_enc, y_prob, labels=self.labels)
#help(log_loss)
bundle['preproc']['le'].classes_.shape

(75,)

In [54]:
X_transformed = bundle['preproc']['one_hot_enc'].transform(X[:1000])
y_prob = model.predict_proba(X_transformed)

In [57]:
y_prob[0].shape

(54,)

In [65]:
# ok got to clear up this inconsistency.. so, len(bundle['preproc']['one_hot_enc'].categories[0]) => 75
# but there are I believe only 54 neighborhoods actually represented in the data itself,

from collections import Counter
print(len(bundle['preproc']['one_hot_enc'].categories[0]))
print(len(dict(Counter([x[0] for x in X]))))

75
54


In [83]:
sorted(list(dict(Counter(y)).keys()))

['Alphabet City',
 'Battery Park City',
 'Bedford-Stuyvesant',
 'Boerum Hill',
 'Bowery',
 'Brooklyn Heights',
 'Brooklyn Navy Yard',
 'Central Park',
 'Chelsea',
 'Chinatown',
 'Civic Center',
 'Clinton Hill',
 'Columbia Street Waterfront District',
 'Downtown Brooklyn',
 'Dumbo',
 'East Village',
 'Financial District',
 'Flatiron District',
 'Fort Greene',
 'Fulton Ferry District',
 'Garment District',
 'Gramercy Park',
 'Greenwich Village',
 "Hell's Kitchen",
 'Hudson Square',
 'Kips Bay',
 'Korea Town',
 'Lincoln Square',
 'Little Italy',
 'Lower East Side',
 'Lower Manhattan',
 'Meatpacking District',
 'Midtown',
 'Midtown East',
 'Midtown West',
 'Murray Hill',
 'NoHo',
 'NoMad',
 'Nolita',
 'Park Slope',
 'Peter Cooper Village',
 'Rose Hill',
 'SoHo',
 'Stuyvesant Town',
 'Sutton Place',
 'Theater District',
 'Tribeca',
 'Tudor City',
 'Two Bridges',
 'Ukrainian Village',
 'Union Square',
 'Vinegar Hill',
 'West Village',
 'Williamsburg']

In [ ]:
# yea so 54 in X and y... but i used stationsdf w/ 75 neighborhoods, 
# so at the very least I should not be using that if it ends up indeed , 
# blowing up the one hot encoder to 75 , ... this might be why we have those start neighborhoods, 
# with  0 importance, since there was just no data for them 

# Anyhow since the predict_proba output is indeed only length 54, that tells me the classifier
# ended up doing the rigiht thing anway. So I will keep this in mind for the next dataset but 
# in this case it is okay anyway.

### 2020-06-13

#### quick update, also just calculate a multiclass logloss as well
- Hmm don't have the indices I used in notebook "2020-06-10-again" so I have to do that again
- Also kernel died while trying to do this so numbers starting over 
- And wow happened twice. Turns out running log_loss() with 843416 rows did the trick!

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [17]:
# load model from 2020-06-10 notebook
# bundle = joblib.load('/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib')

```
example
log_loss(y[:100], y_prob_vec[:100], labels=sorted(list(dict(Counter(y)).keys())))
=> 4.133

losses_vec = []
for part in fu.get_partitions(list(range(len(y_prob_vec))), slice_size=1000):
    i, j = part[0], part[-1]   
    losses_vec.append(log_loss(y[i:j], y_prob_vec[i:j], labels=labels))

fu.big_logloss(y, y_prob=y_prob_vec, labels=labels)
```

#### kernel keeps dying when I'm trying to do this
* I finally started to log some breadcrumbs and I am seeing now where it is dying.. 
* I see that it is dying right before the second `xgb.XGBClassifier()` 
```
2020-06-14 17:49:40Z, Starting
2020-06-14 17:49:44Z, [0] Done preprocessing
2020-06-14 18:01:09Z, [0] Done fit
2020-06-14 18:01:09Z, [0] Done transforming test data
2020-06-14 18:02:27Z, [0] Done predict()
2020-06-14 18:03:48Z, [0] Done  predict_proba()
2020-06-14 18:03:48Z, [0] Done  done fu.big_logloss()
2020-06-14 18:03:48Z, [0] wrote bundle /opt/program/artifacts/2020-06-14T174940Z/bundle_0.joblib
2020-06-14 18:03:50Z, [1] Done preprocessing
```
* The `xgb.XGBClassifier()` is indeed here the longest segment. I am now wondering about borrowing a technique from another project where I would just call a subprocess 
* Reading about using [external memory](https://xgboost.readthedocs.io/en/latest/tutorials/external_memory.html)
* Maybe I can contribute to [this stackoverflow question](https://stackoverflow.com/questions/43972009/how-to-load-a-big-train-csv-for-xgboost) . And indeed [this np.memmap](https://stackoverflow.com/questions/16149803/working-with-big-data-in-python-and-numpy-not-enough-ram-how-to-save-partial-r/16633274#16633274) answer is interesting too.


#### I ended up trying out the external memory approach
- Based on what I read at least, I could only find this within the scope of the non-scikit learn implementation of XG boost. The 
- Or perhaps I can still keep trying, but for the below approach, it was the native `xgb.train()` usage.

#### But this setup is using some different set of parameters and is doing terribly
- My `('/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib')` bundle being inspected at the head of this notebook has an accuracy of about `0.15` and I do not have a logloss estimate yet.

In [34]:
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [41]:
dtest = xgb.DMatrix(f'{outpath}#dtest.cache')

[23:14:46] SparsePageSource::CreateRowPage Finished writing to dtest.cache
[23:14:47] 8000x78 matrix with 8000 entries loaded from /opt/program/artifacts/2020-06-14T225321Z/dtest.0.test#dtest.cache


In [33]:
fu.save_libsvm(X_transformed.toarray(), y_enc, outpath)
fu.log(workdir, f'[{i}] Saved train data: {outpath}')
dtrain = xgb.DMatrix(f'{outpath}#dtrain.cache')

param = {'max_depth':2, 'eta':1, 'objective':'multi:softprob', 'num_class': 54}


watchlist = [ (dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)



[23:04:00] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-14T225321Z/dtrain.0.train#dtrain.cache
[23:04:00] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[23:04:00] SparsePageSource: Finished writing to dtrain.cache
[0]	train-merror:1
[1]	train-merror:1


In [46]:
y_prob = bst.predict(dtest)
y_prob_classes = np.argmax(y_prob, axis=1)

In [48]:
y_prob_classes[:5]

array([0, 0, 0, 0, 0])

#### Another go
But store the intermediary results this time unlike inthe 2020-06-10 notebook

In [28]:
help(xgb.XGBClassifier().fit) # feature_names=[], label=? 


Help on method fit in module xgboost.sklearn:

fit(X, y, sample_weight=None, eval_set=None, eval_metric=None, early_stopping_rounds=None, verbose=True, xgb_model=None, sample_weight_eval_set=None, callbacks=None) method of xgboost.sklearn.XGBClassifier instance
    Fit gradient boosting classifier
    
    Parameters
    ----------
    X : array_like
        Feature matrix
    y : array_like
        Labels
    sample_weight : array_like
        Weight for each instance
    eval_set : list, optional
        A list of (X, y) pairs to use as a validation set for
        early-stopping
    sample_weight_eval_set : list, optional
        A list of the form [L_1, L_2, ..., L_n], where each L_i is a list of
        instance weights on the i-th validation set.
    eval_metric : str, callable, optional
        If a str, should be a built-in evaluation metric to use. See
        doc/parameter.rst. If callable, a custom evaluation metric. The call
        signature is func(y_predicted, y_true) wh

#### 

In [3]:
#%%time

def runner():

    # New workdir
    workdir = fu.make_work_dir(); print('workdir: ', workdir)
    fu.log(workdir, 'Starting')

    rng = np.random.RandomState(31337)
    X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
    neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
    labels = sorted(list(dict(Counter(y)).keys()))

    #X, y = X[:10000], y[:10000]

    kf = KFold(n_splits=5, shuffle=True, random_state=rng)
    for i, (train_index, test_index) in enumerate(tqdm(kf.split(X), desc='outer', leave=True)):

        # preprocess
        (X_transformed, one_hot_enc, le,
             y_enc) = pv1.preprocess(X[train_index], y[train_index], 
                                 neighborhoods)
        fu.log(workdir, f'[{i}] Done preprocessing')
        labels = le.classes_
        param = {'max_depth':3, 'eta': .1, 'objective':'multi:softprob', 'num_class': len(labels)}  # 54


        #
        outpath = f'{workdir}/dtrain.txt'
        fu.save_libsvm(X_transformed.toarray(), y_enc, outpath)
        fu.log(workdir, f'[{i}] Saved train data: {outpath}')
        dtrain = xgb.DMatrix(f'{outpath}#dtrain.cache')
        #xgb_model = xgb.XGBClassifier().fit(dtrain) #X_transformed, y_enc
        watchlist = [ (dtrain, 'train')]
        num_round = 2
        
        xgb_model = xgb.train(param, dtrain, num_round, watchlist)

        fu.log(workdir, f'[{i}] Done fit')

        X_test_transformed = one_hot_enc.transform(X[test_index]).toarray()
        actuals = le.transform(y[test_index])
        fu.log(workdir, f'[{i}] Done transforming test data')

        outpath = f'{workdir}/dtest.txt'    
        fu.save_libsvm(X_test_transformed, outpath=outpath)
        fu.log(workdir, f'[{i}] Done saving to {outpath}')
        dtest = xgb.DMatrix(f'{outpath}') # #dtest.cache


        # predictions = xgb_model.predict(X_test_transformed)
        y_prob_vec = xgb_model.predict(dtest)
        predictions = np.argmax(y_prob_vec, axis=1)

        fu.log(workdir, f'[{i}] Done predict()')
        correct = len([i for i, _ in enumerate(actuals)
                  if actuals[i] == predictions[i]])

        #y_prob_vec = []
        #X_parts = fu.get_partitions(X_test_transformed, slice_size=1000)
        #for X_part in tqdm(X_parts, desc='inner', leave=False):
        #    # X_transformed = one_hot_enc.transform(X_part)
        #    y_prob = xgb_model.predict_proba(X_part)
        #    y_prob_vec.extend(y_prob)
        #fu.log(workdir, f'[{i}] Done  predict_proba()')
        # y_prob_vec = xgb_model.predict_proba(X_test_transformed)

        bundle_file = f'{workdir}/bundle_{i}.joblib'
        loss = fu.big_logloss(actuals, y_prob=y_prob_vec, labels=labels)
        fu.log(workdir, f'[{i}] Done  done fu.big_logloss()')

        joblib.dump({'loss': loss,
         'confusion_matrix': confusion_matrix(actuals, predictions),
         'model': xgb_model,
         'notebook': '2020-06-12.ipynb',
         'accuracy': correct/len(actuals),
         'timestamp': fu.utc_ts(),
        }, bundle_file)
        fu.log(workdir, f'[{i}] wrote bundle {bundle_file}')
        
import ipdb
#ipdb.runcall(runner)
runner()

workdir:  /opt/program/artifacts/2020-06-15T003722Z


[00:38:24] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtrain.txt#dtrain.cache
[00:38:24] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[00:38:24] SparsePageSource: Finished writing to dtrain.cache
[0]	train-merror:1
[1]	train-merror:1
[00:38:39] 168684x78 matrix with 168684 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtest.txt
[00:39:41] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtrain.txt#dtrain.cache
[00:39:41] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[00:39:41] SparsePageSource: Finished writing to dtrain.cache
[0]	train-merror:1
[1]	train-merror:1
[00:39:56] 168683x78 matrix with 168683 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtest.txt
[00:40:56] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/202

In [7]:
workdir = '/opt/program/artifacts/2020-06-15T003722Z/' # artifacts/2020-06-15T003722Z/bundle_0.joblib
bundle = joblib.load(f'{workdir}/bundle_2.joblib')
bundle

{'loss': 0.0,
 'confusion_matrix': array([[3738,    0,    0, ...,    0,    0,    0],
        [4807,    0,    0, ...,    0,    0,    0],
        [ 931,    0,    0, ...,    0,    0,    0],
        ...,
        [ 170,    0,    0, ...,    0,    0,    0],
        [7305,    0,    0, ...,    0,    0,    0],
        [2473,    0,    0, ...,    0,    0,    0]]),
 'model': <xgboost.core.Booster at 0x7f8b60a7b190>,
 'notebook': '2020-06-12.ipynb',
 'accuracy': 0.022159909415886605,
 'timestamp': '2020-06-15T004112Z'}

In [5]:
bundle

{'loss': 0.0,
 'confusion_matrix': array([[3774,    0,    0, ...,    0,    0,    0],
        [4818,    0,    0, ...,    0,    0,    0],
        [ 913,    0,    0, ...,    0,    0,    0],
        ...,
        [ 187,    0,    0, ...,    0,    0,    0],
        [7111,    0,    0, ...,    0,    0,    0],
        [2364,    0,    0, ...,    0,    0,    0]]),
 'model': <xgboost.core.Booster at 0x7f8b60adb410>,
 'notebook': '2020-06-12.ipynb',
 'accuracy': 0.022373194849541155,
 'timestamp': '2020-06-15T003840Z'}

In [12]:
bundle['model'].get_fscore()#(f'{workdir}/model_0.json', with_stats=True, dump_format='json')

{}

In [61]:
dtest.num_row(), dtest.num_col() #    y_prob_vec = xgb_model.predict(dtest)
# This is too weird.
'''
ipdb> p xgb.DMatrix('/opt/program/artifacts/2020-06-14T235247Z/dtest.txt').num_row()
[00:05:44] 2000x78 matrix with 2000 entries loaded from /opt/program/artifacts/2020-06-14T235247Z/dtest.txt
2000
ipdb> p xgb.DMatrix('/opt/program/artifacts/2020-06-14T235247Z/dtest.txt#dtest.cache').num_row()
[00:06:31] 8000x78 matrix with 8000 entries loaded from /opt/program/artifacts/2020-06-14T235247Z/dtest.txt#dtest.cache
8000
'''

(8000, 78)

In [ ]:
# 0:45 , starting 
# 1:00 ,  still going, but dang no sign of the inner tqdm yet (the predict proba)
# 1:01 => TypeError, sparse matrix length is ambiguous; use getnnz() or shape[0]
#          
#/opt/program/fresh/utils.py in get_partitions(vec, slice_size, keep_remainder)
#      39 def get_partitions(vec, slice_size, keep_remainder=True):
#      40     assert slice_size > 0
# ---> 41     num_slices = int(math.floor(len(vec)/slice_size))

In [26]:
# fu.save_libsvm(X_transformed, y_enc, outpath)
X_transformed.toarray()[0][0]

0.0

In [21]:
from tqdm.notebook import tqdm
import time
for i in tqdm(range(10), desc='outer', leave=True):
    for j in tqdm(range(10), desc='inner', leave=False):
        time.sleep(.5)


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


foo
foo
foo
foo
foo
foo
foo
foo
foo
foo



In [24]:
# pretty sure since I had had to partition the logloss , I can still average it to get a full score.
i, j = 0, 1000
a = log_loss(y[i:j], y_prob_vec[i:j], labels=labels)

i, j = 1000, 2000
b = log_loss(y[i:j], y_prob_vec[i:j], labels=labels)

i, j = 0, 2000
c = log_loss(y[i:j], y_prob_vec[i:j], labels=labels)

(a+b)/2, c


(3.993434763431549, 3.9934347634315492)

#### Future food for thought
- [here](https://medium.com/@gabrielziegler3/multiclass-multilabel-classification-with-xgboost-66195e4d9f2d) , a note about using one versus rest approach 
